Importamos librerias

In [79]:
import pandas as pd
import numpy as np

Importamos los csv

In [29]:
df_reviews = pd.read_csv("Datasets/reviews_ETL.csv")
df_items = pd.read_csv("Datasets/user_items_EtL.csv")
df_steam_games = pd.read_csv("Datasets/steam_games_API.csv")

1. def developer( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrollador

In [87]:
def developer_data(desarrollador: str):
    df_dev = df_steam_games.loc[:,['developer', 'item_id', 'price','release_date']]#Sacamos del Dataframe las columnas a necesitar
    dev_desarrolladora = df_dev[df_dev['developer'] == desarrollador]#Se filtra el DataFrame para los juegos de la empresa desarrolladora especifica
    juegos_total = dev_desarrolladora.groupby('release_date').size().reset_index(name='Cantidad de Items')#Agrupamos por año y cuenta la cantidad de juegos
    juegos_free = dev_desarrolladora[dev_desarrolladora['price'] == 0]#Filtra los juegos free
    juegos_freeanio = juegos_free.groupby('release_date').size().reset_index(name='Juegos_cantidad')#Agrupa los juegos free por año y cuenta el total
    resultado = pd.merge(juegos_total,juegos_freeanio, on='release_date', how='left')#Junta los datos de juegos totales y juegos free por año
    resultado['Juegos_cantidad'].fillna(0, inplace=True)#En los valores nulos en juegos_cantidad se coloca 0
    resultado['porcentaje_juegos_gratuitos'] = (resultado['Juegos_cantidad'] / resultado['Cantidad de Items']) * 100#Se calcula el porcentaje de juegos gratuitos por año
    resultado.drop(columns='Juegos_cantidad', inplace=True)#Se elimian la columna 'juegos_cantidad'
    resultado.rename(columns={'release_date': 'Año', 'porcentaje_juegos_gratuitos': 'Contenido Free'}, inplace=True)#Renombramos las columnas como en las indicaciones
    resultado['Cantidad de Items'] = resultado['Cantidad de Items'].astype(int)#Convierte la columna 'Cantidad de Items' a enteros 'Contenido Free' a flotantes
    resultado['Contenido Free'] = resultado['Contenido Free'].apply(lambda x: '{:.2f}%'.format(x))#'Contenido Free' a flotantes
    #Convierte el DataFrame a una lista de diccionarios como se indica
    resultado_final_dev = resultado
    return resultado_final_dev


In [88]:
developer_data("Kotoshiro")

,Año,Cantidad de Items,Contenido Free
0,2018,5,0.00%


In [89]:
developer_data("Poolians.com")

,Año,Cantidad de Items,Contenido Free
0,2017,5,0.00%


2. def userdata( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

Ejemplo de retorno: {"Usuario X" : us213ndjss09sdf, "Dinero gastado": 200 USD, "% de recomendación": 20%, "cantidad de items": 5}

In [47]:
def userdata(user_id: str):
  user_id = str(user_id)# Convertimos el user_id a cadena para asegurar coincidencia en las comparaciones
  user_compras = df_items[df_items['user_id'] == user_id]# Filtrar las compras del usuario en df_items
  user_compras = pd.merge(user_compras, df_steam_games, left_on='item_id', right_on='item_id', how='inner')# Combinamos la información de las compras con los datos de los juegos en df_steam_games
  user_compras['price'] = pd.to_numeric(user_compras['price'], errors='coerce')# Asegurarnos que "price" sea tratada como tipo numerico (para evitar errores de suma)
  total_price = round(user_compras['price'].sum(), 2)# Calculamos el monto de gasto total por usuarios
  user_reviews = df_reviews[(df_reviews['user_id'] == user_id) & (df_reviews['item_id'].isin(user_compras['item_id']))] # Se filtran las revisiones del usuario en df_reviews
  recomendacion_posi = (user_reviews['recommend'].sum() / float(len(user_reviews))) * 100 if len(user_reviews) > 0 else 0# Se calcula el porcentaje de recomendaciones positiva
  recomendacion_posi = f"{recomendacion_posi:.2f}%"
  items_count = len(user_compras)# Calcular la cantidad de ítems comprados
  user_name = user_compras['user_id'].iloc[0] if not user_compras.empty else None# Extraer el nombre del usuario

  # Devolver las estadísticas
  return {
    'Usuarios': user_name,
    'Dinero gastado': total_price,
    'Porcentaje de recomendación': recomendacion_posi,
    'Cantidad de Ítems': items_count
  }


In [48]:
userdata("76561198054465623")

{'Usuarios': '76561198054465623',
 'Dinero gastado': 2716.41,
 'Porcentaje de recomendación': '100.00%',
 'Cantidad de Ítems': 610}

In [49]:
userdata("76561198074257942")

{'Usuarios': '76561198074257942',
 'Dinero gastado': 4309.99,
 'Porcentaje de recomendación': '0.00%',
 'Cantidad de Ítems': 345}

In [51]:
userdata(76561197970982479)

{'Usuarios': '76561197970982479',
 'Dinero gastado': 6497.29,
 'Porcentaje de recomendación': '100.00%',
 'Cantidad de Ítems': 451}

3. def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.

Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [37]:
def UserForGenre(genero: str):
    filtro_steam = df_steam_games[df_steam_games["genres"].str.contains(genero, case=False, na=False)]# Filtra df_steam_games por el género especificado
    filtro_items = df_items[df_items["item_id"].isin(filtro_steam["item_id"])][["item_id", "user_id", "playtime_forever"]]# Filtra df_items por los juegos del género y selecciona las columnas necesarias
    df_joined = pd.merge(filtro_steam, filtro_items[['item_id', 'playtime_forever','user_id']], on='item_id', how='inner')# Se hace un join entre df_games y df_UserItems utilizando 'item_id' como clave
    total_hours_by_user_and_year = df_joined.groupby(['user_id', 'release_date'])['playtime_forever'].sum()# Se agrupa por usuario y año, sumamos las horas jugadas y se encuentra el usuario con más horas jugadas para el género dado
    max_user = total_hours_by_user_and_year.groupby('user_id').sum().idxmax()

    # Se obtiene la lista de acumulación de horas jugadas por año para el usuario con más horas jugadas
    max_user_hours_by_year = total_hours_by_user_and_year.loc[max_user].reset_index()
    max_user_hours_list = [{"Año": int(row['release_date']), "Horas": row['playtime_forever']} for _, row in max_user_hours_by_year.iterrows()]
    # Se retorna el resultado en un formato específico
    result = {"Usuario con más horas jugadas para Género {}".format(genero): max_user, "Horas jugadas": max_user_hours_list}
    return result


In [38]:
UserForGenre("Indie")

{'Usuario con más horas jugadas para Género Indie': 'Efreak',
 'Horas jugadas': [{'Año': 0, 'Horas': 543},
  {'Año': 1997, 'Horas': 154},
  {'Año': 1998, 'Horas': 0},
  {'Año': 2001, 'Horas': 0},
  {'Año': 2002, 'Horas': 0},
  {'Año': 2003, 'Horas': 12},
  {'Año': 2004, 'Horas': 1381},
  {'Año': 2005, 'Horas': 27},
  {'Año': 2006, 'Horas': 1209},
  {'Año': 2007, 'Horas': 1660},
  {'Año': 2008, 'Horas': 6757},
  {'Año': 2009, 'Horas': 24482},
  {'Año': 2010, 'Horas': 28654},
  {'Año': 2011, 'Horas': 285127},
  {'Año': 2012, 'Horas': 39630},
  {'Año': 2013, 'Horas': 24350},
  {'Año': 2014, 'Horas': 39951},
  {'Año': 2015, 'Horas': 62127},
  {'Año': 2016, 'Horas': 19784},
  {'Año': 2017, 'Horas': 837},
  {'Año': 2018, 'Horas': 0}]}

In [13]:
UserForGenre("Simulation")

{'Usuario con más horas jugadas para Género Simulation': 'thequeenpanda',
 'Horas jugadas': [{'Año': 2001, 'Horas': 152},
  {'Año': 2006, 'Horas': 255613},
  {'Año': 2007, 'Horas': 1088},
  {'Año': 2008, 'Horas': 462},
  {'Año': 2009, 'Horas': 497},
  {'Año': 2010, 'Horas': 10026},
  {'Año': 2011, 'Horas': 2487},
  {'Año': 2012, 'Horas': 992},
  {'Año': 2013, 'Horas': 59267},
  {'Año': 2014, 'Horas': 4857},
  {'Año': 2015, 'Horas': 6176},
  {'Año': 2016, 'Horas': 6212},
  {'Año': 2017, 'Horas': 2670}]}

In [14]:
UserForGenre("Casual")

{'Usuario con más horas jugadas para Género Casual': '76561197994136663',
 'Horas jugadas': [{'Año': 2012, 'Horas': 0},
  {'Año': 2015, 'Horas': 230259},
  {'Año': 2017, 'Horas': 57}]}

4. def best_developer_year( año : int ): Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)

Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [39]:
def best_developer_year(year: int):
  df_steam_games['release_date'] = df_steam_games['release_date'].astype(str)#Convierte la columna release_date a cadena de texto
  juegos_del_año = df_steam_games[df_steam_games['release_date'].str[:4] == str(year)]#Filtramos los juegos del año especificado
  juegos_recomendados = pd.merge(juegos_del_año, df_reviews, left_on="item_id", right_on="item_id", how="inner")#Se Combinan los DataFrames para obtener juegos recomendados
  juegos_recomendados = juegos_recomendados[(juegos_recomendados["recommend"] == True) & (juegos_recomendados["sentiment_analisis"] == 2)] #Filtramos juegos recomendados con comentarios positivo
  #Agrupamos por desarrollador y contamos las recomendaciones
  desarrolladores_recomendados = juegos_recomendados["developer"].value_counts().reset_index()
  desarrolladores_recomendados.columns = ["developer", "recommend_count"]
  top_desarrolladores = desarrolladores_recomendados.nlargest(3, "recommend_count")#Ordenamos por recomendaciones y tomar los 3 mejores
  #Formatear el resultado como una lista de diccionarios
  resultado = [
    {"Puesto {}: {}".format(i + 1, row["developer"]): row["recommend_count"]}
    for i, row in top_desarrolladores.iterrows()
  ]

  return resultado


In [40]:
best_developer_year("2017")

[{'Puesto 1: Smartly Dressed Games': 1260},
 {'Puesto 2: Freejam': 456},
 {'Puesto 3: Studio Wildcard,Instinct Games,Efecto Studios,Virtual Basement LLC': 292}]

In [41]:
best_developer_year("2008")

[{'Puesto 1: CD PROJEKT RED': 20},
 {'Puesto 2: Rockstar New England': 18},
 {'Puesto 3: Treyarch': 18}]

5. def developer_reviews_analysis( desarrolladora : str ): Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.

Ejemplo de retorno: {'Valve' : [Negative = 182, Positive = 278]}

In [42]:
def developer_reviews_analysis(desarrolladora: str):
    juegos_desarrollador = df_steam_games[df_steam_games["developer"] == desarrolladora]#Filtramoss los juegos del desarrollador
    reseñas_desarrollador = df_reviews[df_reviews["item_id"].isin(juegos_desarrollador["item_id"])]#Filtramos las reseñas por los juegos del desarrollador
    # Cuenta las reseñas con sentimiento positivo y negativo
    sentimiento_positivo = reseñas_desarrollador[reseñas_desarrollador["sentiment_analisis"] == 2].shape[0]
    sentimiento_negativo = reseñas_desarrollador[reseñas_desarrollador["sentiment_analisis"] == 0].shape[0]
    # Crea el diccionario de resultados
    resultado = {
        desarrolladora: {
            "Positive": sentimiento_positivo,
            "Negative": sentimiento_negativo,
        }
    }

    return resultado



In [61]:
developer_reviews_analysis("3000AD")

{'3000AD': {'Positive': 0, 'Negative': 1}}

In [62]:
developer_reviews_analysis("iWin")

{'iWin': {'Positive': 0, 'Negative': 0}}

In [63]:
developer_reviews_analysis("SEGA")

{'SEGA': {'Positive': 12, 'Negative': 5}}